In [22]:
import numpy as np
import pandas as pd
import mysql.connector

In [23]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="1234"
)

print(mydb)
mycursor = mydb.cursor()
mycursor.execute("use csgo_data")
# mycursor.execute("TRUNCATE TABLE player_stats")


In [24]:
df = pd.read_csv("../data-collecter-js/newOut.csv")
df.head(5)

,name,country,url,teams,maps_played,rounds_played,kd_difference,kd_ratio,rating,total_kills,image_url
0,torzsi,Hungary,https://www.hltv.org/stats/players/18072/torzsi,"MOUZ,Budapest Five,MOUZ NXT,Salamander",476,12723,2008,1.26,1.16,9868,https://img-cdn.hltv.org/playerbodyshot/vXrIz7...
1,sh1ro,Russia,https://www.hltv.org/stats/players/16920/sh1ro,"Gambit Youngsters,Gambit",884,23451,5599,1.45,1.23,18091,https://img-cdn.hltv.org/playerbodyshot/Eyk16c...
2,degster,Russia,https://www.hltv.org/stats/players/17306/degster,"Espada,Spirit,Spirit",686,18284,3363,1.31,1.19,14351,https://img-cdn.hltv.org/playerbodyshot/bH3qqs...
3,sterling,New Zealand,https://www.hltv.org/stats/players/12591/sterling,"AVANT,Looking For Org,Dire Wolves",861,22023,3960,1.30,1.17,17063,https://img-cdn.hltv.org/playerbodyshot/Rg6dzO...
4,xsepower,Russia,https://www.hltv.org/stats/players/12733/xsepower,"K23,K23,forZe",1264,33946,4637,1.23,1.11,24421,https://img-cdn.hltv.org/playerbodyshot/rJA5Of...


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           16 non-null     object 
 1   country        16 non-null     object 
 2   url            16 non-null     object 
 3   teams          16 non-null     object 
 4   maps_played    16 non-null     int64  
 5   rounds_played  16 non-null     int64  
 6   kd_difference  16 non-null     int64  
 7   kd_ratio       16 non-null     float64
 8   rating         16 non-null     float64
 9   total_kills    16 non-null     int64  
 10  image_url      16 non-null     object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1.5+ KB


In [26]:
df.describe()

,maps_played,rounds_played,kd_difference,kd_ratio,rating,total_kills
count,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000
mean,772.312500,20459.812500,3143.25000,1.261875,1.156250,15593.375000
std,319.131889,8370.463581,1484.54287,0.092967,0.053276,6216.760731
min,404.000000,10701.000000,1042.00000,1.130000,1.090000,8482.000000
25%,471.750000,12575.000000,1997.25000,1.177500,1.110000,9668.250000
50%,691.000000,18465.000000,2868.50000,1.260000,1.155000,14127.500000
75%,918.000000,24339.250000,3971.75000,1.317500,1.192500,19223.750000
max,1387.000000,36436.000000,6164.00000,1.450000,1.270000,26662.000000


In [27]:
df_numeric = df.select_dtypes(include=[np.number])
df_categoric = df.select_dtypes(exclude=[np.number])

df_numeric_cols = list(df_numeric.columns)
df_categoric_cols = list(df_categoric.columns)

In [28]:
def maxAndMinStats(dataframe, cols):
    """Extracting out players with interesting statistics"""
    def maxAndMin(dataf, column):
        return dataf.loc[dataf[column].idxmax()], dataf.loc[dataf[column].idxmin()]

    maxMinList = []

    for col in cols:
        print('Column: ', col)
        res = maxAndMin(dataframe, col)
        max_, min_ = res
        print('Max in ', col, ': ', max_['name'])
        # print('Min in ', col, ': ', min_['name'], '\n')
        # print(max_['image_url'])
        # print(max_[col])
        sql= "insert into player_stats(title, count, name, profile_url) VALUES(%s,%s,%s,%s)"
        values=(col,str(max_[col]),max_['name'],max_['image_url'])
        mycursor.execute(sql,values)
        mydb.commit()
        print(mycursor.rowcount, "record(s) inserted.")
        maxMinList.append(res)

    return maxMinList

In [29]:
maxMinStats = maxAndMinStats(df, df_numeric_cols)

Column:  maps_played
Max in  maps_played :  Twistzz
1 record(s) inserted.
Column:  rounds_played
Max in  rounds_played :  Twistzz
1 record(s) inserted.
Column:  kd_difference
Max in  kd_difference :  ZywOo
1 record(s) inserted.
Column:  kd_ratio
Max in  kd_ratio :  sh1ro
1 record(s) inserted.
Column:  rating
Max in  rating :  ZywOo
1 record(s) inserted.
Column:  total_kills
Max in  total_kills :  Twistzz
1 record(s) inserted.


In [12]:
df = pd.read_csv("../data-collecter-js/newOut.csv")
df.head(5)
mycursor = mydb.cursor()
mycursor.execute("TRUNCATE TABLE player_table")

In [13]:
for ind in df.index:
    print(df['name'][ind], df['country'][ind],df["maps_played"][ind])

torzsi Hungary 476
sh1ro Russia 884
degster Russia 686
sterling New Zealand 861
xsepower Russia 1264
mantuu Poland 696
frozen Slovakia 1216
hades Poland 602
NEOFRAG Czech Republic 676
deko Russia 404
blameF Denmark 880
saffee Brazil 404
Twistzz Canada 1387
ZywOo France 1020
SENSEi Ukraine 459
Spinx Israel 442


In [14]:
for ind in df.index:
    sql= "insert into player_table(name, country, maps_played,rounds_played,kd_ratio,rating,total_kills) VALUES(%s,%s,%s,%s,%s,%s,%s)"
    values=(df['name'][ind],df['country'][ind],str(df["maps_played"][ind]),str(df["rounds_played"][ind]),str(df["kd_ratio"][ind]),str(df["rating"][ind]),str(df["total_kills"][ind]))
    mycursor.execute(sql,values)
    mydb.commit()